In [1]:
import tarfile
import numpy as np
import codecs

In [2]:
tar = tarfile.open("02-14-SOCIAL_MEDIA.tar.gz", "r:gz")
#for member in tar.getmembers():
#    f = tar.extractfile(member)
#    if f is not None:
#        content = f.read()

In [3]:
members = tar.getmembers()

In [4]:
len(members)

35616

In [5]:
x = tar.extractfile(members[4]).read()

# Reading a Single `.protostream` File

In [2]:
import spinn3rApi_pb2 as proto_api
import protoStream_pb2 as proto_stream
from google.protobuf.internal.decoder import _DecodeVarint32

In [3]:
decoder = _DecodeVarint32

## get the three types of protobuf messages we expect to see
header    = proto_stream.ProtoStreamHeader()
delimiter = proto_stream.ProtoStreamDelimiter()
entry     = proto_api.Entry()

In [47]:
data = open('1.protostream', 'rb').read()

In [48]:
len(data)

314204

In [74]:
## get the header
pos = 0
next_pos, pos = decoder(data, pos)
header.ParseFromString(data[pos:pos + next_pos])
print(header)
## should check its contents

pos += next_pos

files = []
pos_diff = []

while pos < len(data):
    next_pos, pos = decoder(data, pos)
    delimiter.ParseFromString(data[pos:pos + next_pos])
    print(pos, next_pos, delimiter)
    if delimiter.delimiter_type == delimiter.END:
        break
    elif delimiter.delimiter_type == delimiter.ENTRY:
        pos += next_pos
        next_pos, pos = decoder(data, pos)
        print(pos, next_pos)
        pos_diff.append(next_pos)
        entry = proto_api.Entry()
        entry.ParseFromString(data[pos:pos + next_pos])
        files.append(entry)
    pos += next_pos

version: "1.0"
default_entry_type: "contentApi.Entry"

25 2 delimiter_type: ENTRY

29 1504
1534 2 delimiter_type: ENTRY

1538 1261
2800 2 delimiter_type: ENTRY

2804 1482
4287 2 delimiter_type: ENTRY

4291 1273
5565 2 delimiter_type: ENTRY

5569 1733
7303 2 delimiter_type: ENTRY

7307 1368
8676 2 delimiter_type: ENTRY

8680 1670
10351 2 delimiter_type: ENTRY

10355 1695
12051 2 delimiter_type: ENTRY

12055 1594
13650 2 delimiter_type: ENTRY

13654 1364
15019 2 delimiter_type: ENTRY

15023 1263
16287 2 delimiter_type: ENTRY

16291 1250
17542 2 delimiter_type: ENTRY

17546 1190
18737 2 delimiter_type: ENTRY

18741 1423
20165 2 delimiter_type: ENTRY

20169 1373
21543 2 delimiter_type: ENTRY

21547 1500
23048 2 delimiter_type: ENTRY

23052 1371
24424 2 delimiter_type: ENTRY

24428 1203
25632 2 delimiter_type: ENTRY

25636 1436
27073 2 delimiter_type: ENTRY

27077 1813
28891 2 delimiter_type: ENTRY

28895 1423
30319 2 delimiter_type: ENTRY

30323 1298
31622 2 delimiter_type: ENTRY

31626 13

In [75]:
len(files)

200

In [51]:
type(files[0])

spinn3rApi_pb2.Entry

In [82]:
files[2]

source {
  link {
    href: ""
    resource: ""
  }
  canonical_link {
    href: "http://facebook.com/post/?id=100001062367788_182859851749721"
    resource: ""
  }
  title: ""
  hashcode: ""
  lang {
    code: "en"
    probability: -1.0
  }
  generator: ""
  description: ""
  last_posted: ""
  last_published: ""
  date_found: ""
  publisher_type: "SOCIAL_MEDIA"
}
feed {
  link {
    href: ""
    resource: ""
  }
  canonical_link {
    href: ""
    resource: ""
  }
  title: ""
  hashcode: ""
  lang {
    code: "en"
    probability: -1.0
  }
  generator: ""
  description: ""
  last_posted: ""
  last_published: ""
  date_found: ""
  etag: ""
  channel_link {
    href: ""
    resource: ""
  }
}
feed_entry {
  link {
    href: "http://facebook.com/post/?id=100001062367788_182859851749721"
    resource: "http://facebook.com/post/?id=100001062367788_182859851749721"
  }
  canonical_link {
    href: "http://facebook.com/post/?id=100001062367788_182859851749721"
    resource: "http://facebook.

In [83]:
titles = [file.feed_entry.title for file in files]

In [95]:
[title for title in titles if 'europe' in title]

[]

In [77]:
lengths = [len(str(file)) for file in files]

In [78]:
set(lengths)

{2490,
 2497,
 2523,
 2539,
 2553,
 2560,
 2570,
 2574,
 2577,
 2581,
 2583,
 2585,
 2589,
 2592,
 2597,
 2600,
 2603,
 2604,
 2620,
 2624,
 2628,
 2632,
 2633,
 2635,
 2636,
 2641,
 2644,
 2648,
 2649,
 2652,
 2654,
 2657,
 2661,
 2669,
 2670,
 2684,
 2688,
 2690,
 2695,
 2696,
 2698,
 2700,
 2701,
 2705,
 2711,
 2714,
 2716,
 2719,
 2722,
 2724,
 2725,
 2734,
 2735,
 2745,
 2750,
 2762,
 2763,
 2765,
 2766,
 2771,
 2773,
 2776,
 2779,
 2788,
 2789,
 2797,
 2798,
 2804,
 2811,
 2812,
 2814,
 2816,
 2819,
 2820,
 2822,
 2823,
 2828,
 2829,
 2832,
 2840,
 2841,
 2843,
 2844,
 2846,
 2850,
 2861,
 2864,
 2866,
 2879,
 2887,
 2889,
 2899,
 2905,
 2909,
 2910,
 2911,
 2912,
 2914,
 2915,
 2916,
 2929,
 2930,
 2934,
 2937,
 2939,
 2948,
 2949,
 2954,
 2956,
 2965,
 2967,
 2968,
 2979,
 2984,
 2994,
 2998,
 3014,
 3015,
 3017,
 3021,
 3023,
 3024,
 3026,
 3030,
 3032,
 3034,
 3041,
 3047,
 3058,
 3059,
 3064,
 3081,
 3088,
 3095,
 3107,
 3113,
 3114,
 3116,
 3122,
 3125,
 3155,
 3170,
 3188,